In [89]:
import re

def count_cocycles(G): # Get the cohomology as a string
    cohomology_str = str(G) # Use a regular expression to find the contents within the curly braces
    match = re.search(r'{(.*)} over Rational Field', cohomology_str)
    if match: # Retrieve the contents within the curly braces
        cocycles_str = match.group(1) # Split the string into individual cocycles
        cocycles = cocycles_str.split(', ') # Return the count
        return len(cocycles)
    return 0


def classify_cocycles(C): # This classifies the elements in the THIRD cohomology of a Lie algerbas assocaited with a graph
    output = str(C) # Extract the generators
    generators = re.findall(r'\[(.*?)\]', output) # Different lists to hold each type
    P = []  #  P: x1*x2*x3
    A1 = [] # A1: x1*x2*y3_4
    A2 = [] # A2: x1*x2*y3_4 +- x5*x6*y7_8
    B1 = [] # B1: x1*y2_3*y4_5
    B2 = [] # B2: x1*y2_3*y4_5 +- x6*y7_8*y9_10
    B3 = [] # B3: x1*y2_3*y4_5 +- x6*y7_8*y9_10 +- x11*y12_13*y14_15
    B4 = [] # B4: x1*y2_3*y4_5 +- x6*y7_8*y9_10 +- x11*y12_13*y14_15 +- x16*y17_18*y19_20
    B5 = [] # B5: x1*y2_3*y4_5 +- x6*y7_8*y9_10 +- x11*y12_13*y14_15 +- x16*y17_18*y19_20 +- x21*y22_23*y24_25
    B6 = [] # B6: x1*y2_3*y4_5 +- 2*x6*y7_8*y9_10 +- x11*y12_13*y14_15 +- x16*y17_18*y19_20
    for gen in generators: # Check if it's a linear combination
        if '-' in gen or '+' in gen:
            terms = re.split('[-+]', gen.strip())
            term_count = len(terms)
            y_counts = [term.count('y') for term in terms]
            scalar_flag = any(re.search(r'^[2-9]\*', term.strip()) for term in terms)
            if scalar_flag and term_count == 4:
                B6.append(gen)
            elif term_count == 2 and max(y_counts) == min(y_counts) == 1:
                A2.append(gen)
            elif term_count == 2 and max(y_counts) == min(y_counts) == 2:
                B2.append(gen)
            elif term_count == 3:
                B3.append(gen)
            elif term_count == 4 and not scalar_flag:
                B4.append(gen)
            elif term_count == 5:
                B5.append(gen)
        else:
            y_count = gen.count('y')
            if y_count == 0:
                P.append(gen)
            elif y_count == 1:
                A1.append(gen)
            else:
                B1.append(gen) 
    dim_P = len(P) # Compute the counts
    dim_A1 = len(A1)
    dim_A2 = len(A2)
    dim_B1 = len(B1)
    dim_B2 = len(B2)
    dim_B3 = len(B3)
    dim_B4 = len(B4)
    dim_B5 = len(B5)
    dim_B6 = len(B6) 
    total_count = dim_P + dim_A1 + dim_A2 + dim_B1 + dim_B2 + dim_B3 + dim_B4 + dim_B5 + dim_B6
    print("______________________________________")
    if count_cocycles(C) == total_count:
        print("Cocycle Classification Successful") # This statement should be removed if/when these classes are proven to be the only generators in H^3
    else:
        print("Cocycle Classification Not Complete")
    print("______________________________________")
    return {
        'P': (P, dim_P),
        'A1': (A1, dim_A1),
        'A2': (A2, dim_A2),
        'B1': (B1, dim_B1),
        'B2': (B2, dim_B2),
        'B3': (B3, dim_B3),
        'B4': (B4, dim_B4),
        'B5': (B5, dim_B5),
        'B6': (B6, dim_B6),
        'total_count': total_count
    }

def graph_to_CDGA(G):
    vertices = list(G.vertices()) # Index the vertices starting from 1
    index_dict = {v: i+1 for i, v in enumerate(vertices)} # Rename the vertices
    G_relabeled = G.relabel(index_dict, inplace=False)
    vertices = list(G_relabeled.vertices()) # Regenerate the vertices and edges
    edges = G_relabeled.edges(labels=False)
    vars_vertices = ['x{}'.format(i) for i in vertices] # Generate variable names for vertices and edges
    vars_edges = ['y{}_{}'.format(e[0], e[1]) for e in edges] 
    vars_all = vars_vertices + vars_edges # Combine variable names
    A = GradedCommutativeAlgebra(QQ, vars_all)# Generate commutative algebra
    edge_dict = {} # Create dictionary with edge variables as keys and products of corresponding vertices as values
    for e in vars_edges:
        i, j = map(int, e[1:].split('_'))  # Parse variable name to find vertex numbers
        edge_dict[A.gen(vars_all.index(e))] = A.gen(vars_all.index('x{}'.format(i))) * A.gen(vars_all.index('x{}'.format(j)))
    B = A.cdg_algebra(edge_dict) # Build the commutative differential graded algebra
    return B

In [94]:
G = graphs.PathGraph(5)
B = graph_to_CDGA(G)
B.cohomology(3)

/var/folders/yt/xg9kjxjs22s712kz_9hz7j4h0000gn/T/ipykernel_14585/1657737355.py:81: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  vertices = list(G.vertices()) # Index the vertices starting from 1
/var/folders/yt/xg9kjxjs22s712kz_9hz7j4h0000gn/T/ipykernel_14585/1657737355.py:84: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  vertices = list(G_relabeled.vertices()) # Regenerate the vertices and edges
/var/folders/yt/xg9kjxjs22s712kz_9hz7j4h0000gn/T/ipykernel_14585/1657737355.py:85: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See https://trac.sagemath.org/27408 for details.
  edges = G_relabeled.edges(labels=False)


Free module generated by {[x1*x3*x5], [x1*x2*y1_2], [x1*x3*y1_2], [x2*x3*y1_2], [x1*x4*y1_2], [x2*x4*y1_2], [x1*x5*y1_2], [x2*x5*y1_2], [x3*x5*y1_2 - x1*x5*y2_3], [x1*x3*y2_3], [x2*x3*y2_3], [x1*x4*y2_3 - x1*x2*y3_4], [x2*x4*y2_3], [x3*x4*y2_3], [x2*x5*y2_3], [x3*x5*y2_3], [x2*y1_2*y2_3], [x1*x3*y3_4], [x1*x4*y3_4], [x2*x4*y3_4], [x3*x4*y3_4], [x1*x5*y3_4 - x1*x3*y4_5], [x2*x5*y3_4 - x2*x3*y4_5], [x3*x5*y3_4], [x4*x5*y3_4], [x3*y2_3*y3_4], [x1*x4*y4_5], [x2*x4*y4_5], [x1*x5*y4_5], [x2*x5*y4_5], [x3*x5*y4_5], [x4*x5*y4_5], [x4*y3_4*y4_5]} over Rational Field

In [58]:
A = graph_to_CDGA(graphs.PetersenGraph())
C = A.cohomology(3)

/var/folders/yt/xg9kjxjs22s712kz_9hz7j4h0000gn/T/ipykernel_14585/2889453610.py:83: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See http://trac.sagemath.org/22349 for details.
  vertices = G.vertices()
/var/folders/yt/xg9kjxjs22s712kz_9hz7j4h0000gn/T/ipykernel_14585/2889453610.py:84: DeprecationWarning: parameter 'sort' will be set to False by default in the future
See https://trac.sagemath.org/27408 for details.
  edges = G.edges()


In [59]:
classify_cocycles(C)

______________________________________
Cocycle Classification Successful
______________________________________


{'P': (['x2*x4*x6',
   'x2*x5*x6',
   'x3*x5*x6',
   'x1*x3*x7',
   'x1*x4*x7',
   'x3*x5*x7',
   'x3*x6*x7',
   'x4*x6*x7',
   'x5*x6*x7',
   'x1*x4*x8',
   'x2*x4*x8',
   'x2*x5*x8',
   'x1*x7*x8',
   'x4*x7*x8',
   'x5*x7*x8',
   'x1*x3*x9',
   'x2*x5*x9',
   'x3*x5*x9',
   'x1*x8*x9',
   'x2*x8*x9',
   'x5*x8*x9',
   'x1*x3*x10',
   'x1*x4*x10',
   'x2*x4*x10',
   'x2*x6*x10',
   'x3*x6*x10',
   'x4*x6*x10',
   'x1*x9*x10',
   'x2*x9*x10',
   'x3*x9*x10'],
  30),
 'A1': (['x1*x2*y1_2',
   'x1*x3*y1_2',
   'x2*x3*y1_2',
   'x1*x4*y1_2',
   'x2*x4*y1_2',
   'x1*x5*y1_2',
   'x2*x5*y1_2',
   'x1*x6*y1_2',
   'x2*x6*y1_2',
   'x1*x7*y1_2',
   'x2*x7*y1_2',
   'x1*x8*y1_2',
   'x2*x8*y1_2',
   'x1*x9*y1_2',
   'x2*x9*y1_2',
   'x1*x10*y1_2',
   'x2*x10*y1_2',
   'x1*x3*y1_5',
   'x1*x4*y1_5',
   'x1*x5*y1_5',
   'x2*x5*y1_5',
   'x3*x5*y1_5',
   'x4*x5*y1_5',
   'x1*x6*y1_5',
   'x5*x6*y1_5',
   'x1*x7*y1_5',
   'x5*x7*y1_5',
   'x1*x8*y1_5',
   'x5*x8*y1_5',
   'x1*x9*y1_5',
   'x5*x9*